In [1]:
import findspark
findspark.init()
findspark.find()
import pyspark
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from pyspark.sql import functions as f
from pyspark.sql.functions import *
from pyspark.sql.types import StructType, IntegerType
from pyspark.sql.functions import col
from pyspark.sql.functions import sum,count
from pyspark.sql.functions import stddev_pop
from pyspark.sql.functions import max
from pyspark.sql.functions import mean ,corr,max, min,stddev_pop ,stddev_samp,covar_pop,covar_samp
spark = SparkSession.builder.getOrCreate()
sc =SparkContext.getOrCreate()

In [2]:
retail = spark.read.option("inferschema",True).option("header",True).csv("retail.csv")
retail.show()

+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|   536365|   85123A|WHITE HANGING HEA...|       6|2010-12-01 08:26:00|     2.55|   17850.0|United Kingdom|
|   536365|    71053| WHITE METAL LANTERN|       6|2010-12-01 08:26:00|     3.39|   17850.0|United Kingdom|
|   536365|   84406B|CREAM CUPID HEART...|       8|2010-12-01 08:26:00|     2.75|   17850.0|United Kingdom|
|   536365|   84029G|KNITTED UNION FLA...|       6|2010-12-01 08:26:00|     3.39|   17850.0|United Kingdom|
|   536365|   84029E|RED WOOLLY HOTTIE...|       6|2010-12-01 08:26:00|     3.39|   17850.0|United Kingdom|
|   536365|    22752|SET 7 BABUSHKA NE...|       2|2010-12-01 08:26:00|     7.65|   17850.0|United Kingdom|
|   536365|    21730|GLASS S

In [7]:
retail.createOrReplaceTempView("df1")


In [8]:
datacube = retail.select("stockcode","quantity","country")
datacube.show()


+---------+--------+--------------+
|stockcode|quantity|       country|
+---------+--------+--------------+
|   85123A|       6|United Kingdom|
|    71053|       6|United Kingdom|
|   84406B|       8|United Kingdom|
|   84029G|       6|United Kingdom|
|   84029E|       6|United Kingdom|
|    22752|       2|United Kingdom|
|    21730|       6|United Kingdom|
|    22633|       6|United Kingdom|
|    22632|       6|United Kingdom|
|    84879|      32|United Kingdom|
|    22745|       6|United Kingdom|
|    22748|       6|United Kingdom|
|    22749|       8|United Kingdom|
|    22310|       6|United Kingdom|
|    84969|       6|United Kingdom|
|    22623|       3|United Kingdom|
|    22622|       2|United Kingdom|
|    21754|       3|United Kingdom|
|    21755|       3|United Kingdom|
|    21777|       4|United Kingdom|
+---------+--------+--------------+
only showing top 20 rows



SLICING DATA

In [ ]:
slice1 = datacube.select("*").filter(col("country") == "France")
slice1.show()

SUB-CUBE DATA

In [ ]:
subcube = datacube.select("*").filter((col("country")=="United Kingdom") | (col("quantity")>'0'))
subcube.show()

In [ ]:
spark.sql("""select * from df1 where country = "United Kingdom" or quantity > '0' """).show()

ROLL-UP

In [ ]:
rollup = datacube.select("stockcode","country",(col("quantity")>'5').alias("positive"))
rollup.show()

In [6]:
spark.sql("""SELECT stockcode, country,sum("quantity") AS Profit FROM df1  GROUP BY ROLLUP(stockcode, country)""").show()

+---------+--------------+------+
|stockcode|       country|Profit|
+---------+--------------+------+
|    22969|United Kingdom|  null|
|    22807|United Kingdom|  null|
|    22508|United Kingdom|  null|
|    21742|United Kingdom|  null|
|    22445|United Kingdom|  null|
|    22478|United Kingdom|  null|
|    85176|United Kingdom|  null|
|    21158|United Kingdom|  null|
|    37461|United Kingdom|  null|
|    21883|          null|  null|
|    21324|          null|  null|
|   85049A|          null|  null|
|    22558|          null|  null|
|    22706|          null|  null|
|   79066K|          null|  null|
|    85038|          null|  null|
|   90211A|          null|  null|
|    22866|United Kingdom|  null|
|    84347|United Kingdom|  null|
|    21823|United Kingdom|  null|
+---------+--------------+------+
only showing top 20 rows



In [7]:
spark.sql("""select stockcode,country,quantity>5 as positive from df1 """).show()

+---------+--------------+--------+
|stockcode|       country|positive|
+---------+--------------+--------+
|   85123A|United Kingdom|    true|
|    71053|United Kingdom|    true|
|   84406B|United Kingdom|    true|
|   84029G|United Kingdom|    true|
|   84029E|United Kingdom|    true|
|    22752|United Kingdom|   false|
|    21730|United Kingdom|    true|
|    22633|United Kingdom|    true|
|    22632|United Kingdom|    true|
|    84879|United Kingdom|    true|
|    22745|United Kingdom|    true|
|    22748|United Kingdom|    true|
|    22749|United Kingdom|    true|
|    22310|United Kingdom|    true|
|    84969|United Kingdom|    true|
|    22623|United Kingdom|   false|
|    22622|United Kingdom|   false|
|    21754|United Kingdom|   false|
|    21755|United Kingdom|   false|
|    21777|United Kingdom|   false|
+---------+--------------+--------+
only showing top 20 rows



ADDING NEW TIME FORMAT SO READING DATA AGAIN

In [8]:
retail1 = spark.read.option("inferschema","true").option("header","true").option("timestampFormat","dd/M/yyyy H:mm").csv("retail.csv")
retail1.show()

+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|   536365|   85123A|WHITE HANGING HEA...|       6|2010-12-01 08:26:00|     2.55|   17850.0|United Kingdom|
|   536365|    71053| WHITE METAL LANTERN|       6|2010-12-01 08:26:00|     3.39|   17850.0|United Kingdom|
|   536365|   84406B|CREAM CUPID HEART...|       8|2010-12-01 08:26:00|     2.75|   17850.0|United Kingdom|
|   536365|   84029G|KNITTED UNION FLA...|       6|2010-12-01 08:26:00|     3.39|   17850.0|United Kingdom|
|   536365|   84029E|RED WOOLLY HOTTIE...|       6|2010-12-01 08:26:00|     3.39|   17850.0|United Kingdom|
|   536365|    22752|SET 7 BABUSHKA NE...|       2|2010-12-01 08:26:00|     7.65|   17850.0|United Kingdom|
|   536365|    21730|GLASS S

In [9]:
retail1.createOrReplaceTempView("df2")


COUNTING ORDERS AT WHICH HOUR

In [10]:
spark.sql("""select hour(InvoiceDate) as InvoiceHours,count(distinct InvoiceNo) as new from df2 group by InvoiceHours order by InvoiceHours """).show()

+------------+---+
|InvoiceHours|new|
+------------+---+
|           8|  6|
|           9| 18|
|          10| 12|
|          11| 13|
|          12| 23|
|          13| 12|
|          14| 21|
|          15| 15|
|          16| 17|
|          17|  6|
+------------+---+



In [11]:
import pyspark.sql.functions as f
result = retail.selectExpr("hour(invoicedate) as new1",("invoiceno")).distinct().groupBy("new1").agg(f.expr("count(invoiceno) as new2")).orderBy("new1")


In [12]:
result.show()

+----+----+
|new1|new2|
+----+----+
|   8|   6|
|   9|  18|
|  10|  12|
|  11|  13|
|  12|  23|
|  13|  12|
|  14|  21|
|  15|  15|
|  16|  17|
|  17|   6|
+----+----+



CLEANING THE DATA OR REPLACING NULL VALUES

In [13]:
datacube = retail.select("country","stockcode","quantity")
datacube = datacube.na.replace([""],["unknown"],"stockcode").na.replace([""],["unknown"],"country").na.drop("any")

How frequently was each product bought in different countries?

In [14]:
aa=spark.sql("""select country,stockcode,sum(quantity) as new  from df2 group by stockcode,country GROUPING SETS ((Country,Stockcode),(Country),(Stockcode),()) ORDER BY Country, Stockcode""")
aa.show()

+-------+---------+-----+
|country|stockcode|  new|
+-------+---------+-----+
|   null|     null|26814|
|   null|    10002|   60|
|   null|    10125|    2|
|   null|    10133|    5|
|   null|    10135|    1|
|   null|    11001|    3|
|   null|   15044B|    1|
|   null|  15056BL|   20|
|   null|   15056N|   50|
|   null|   15056P|   48|
|   null|    16014|   10|
|   null|    16016|   10|
|   null|   16168M|    2|
|   null|    16236|    2|
|   null|    16237|   70|
|   null|    16238|   34|
|   null|   16258A|    1|
|   null|    17003|   39|
|   null|   17011F|    1|
|   null|   17012A|    4|
+-------+---------+-----+
only showing top 20 rows



In [20]:
df4 = aa.coalesce(1)

df4.write.format("csv").option("header","true").save("C:/Users/User/Desktop/result.csv")

Py4JJavaError: An error occurred while calling o111.save.
: org.apache.spark.SparkException: Job aborted.
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$.write(FileFormatWriter.scala:231)
	at org.apache.spark.sql.execution.datasources.InsertIntoHadoopFsRelationCommand.run(InsertIntoHadoopFsRelationCommand.scala:188)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult$lzycompute(commands.scala:108)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult(commands.scala:106)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.doExecute(commands.scala:131)
	at org.apache.spark.sql.execution.SparkPlan.$anonfun$execute$1(SparkPlan.scala:180)
	at org.apache.spark.sql.execution.SparkPlan.$anonfun$executeQuery$1(SparkPlan.scala:218)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:151)
	at org.apache.spark.sql.execution.SparkPlan.executeQuery(SparkPlan.scala:215)
	at o

CUBE FUNCTION (alternative of group by)

In [318]:
result = datacube.cube("country","stockcode").agg(f.sum("quantity").alias("quantity")).orderBy(f.col("country"),f.col("stockcode"))
result.show()

+-------+---------+--------+
|country|stockcode|quantity|
+-------+---------+--------+
|   null|     null|   26814|
|   null|    10002|      60|
|   null|    10125|       2|
|   null|    10133|       5|
|   null|    10135|       1|
|   null|    11001|       3|
|   null|   15044B|       1|
|   null|  15056BL|      20|
|   null|   15056N|      50|
|   null|   15056P|      48|
|   null|    16014|      10|
|   null|    16016|      10|
|   null|   16168M|       2|
|   null|    16236|       2|
|   null|    16237|      70|
|   null|    16238|      34|
|   null|   16258A|       1|
|   null|    17003|      39|
|   null|   17011F|       1|
|   null|   17012A|       4|
+-------+---------+--------+
only showing top 20 rows



In [323]:
result.coalesce(1).write.format("csv").option("header","true").save("C:/Users/User/Desktop/result.csv")

Py4JJavaError: An error occurred while calling o2054.save.
: org.apache.spark.SparkException: Job aborted.
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$.write(FileFormatWriter.scala:231)
	at org.apache.spark.sql.execution.datasources.InsertIntoHadoopFsRelationCommand.run(InsertIntoHadoopFsRelationCommand.scala:188)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult$lzycompute(commands.scala:108)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult(commands.scala:106)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.doExecute(commands.scala:131)
	at org.apache.spark.sql.execution.SparkPlan.$anonfun$execute$1(SparkPlan.scala:180)
	at org.apache.spark.sql.execution.SparkPlan.$anonfun$executeQuery$1(SparkPlan.scala:218)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:151)
	at org.apache.spark.sql.execution.SparkPlan.executeQuery(SparkPlan.scala:215)
	at 

In [290]:
a= retail.where(col("country")=="Germany")
a.show()

+---------+---------+--------------------+--------+-------------------+---------+----------+-------+
|InvoiceNo|StockCode|         Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|Country|
+---------+---------+--------------------+--------+-------------------+---------+----------+-------+
|   536527|    22809|SET OF 6 T-LIGHTS...|       6|2010-12-01 13:04:00|     2.95|   12662.0|Germany|
|   536527|    84347|ROTATING SILVER A...|       6|2010-12-01 13:04:00|     2.55|   12662.0|Germany|
|   536527|    84945|MULTI COLOUR SILV...|      12|2010-12-01 13:04:00|     0.85|   12662.0|Germany|
|   536527|    22242|5 HOOK HANGER MAG...|      12|2010-12-01 13:04:00|     1.65|   12662.0|Germany|
|   536527|    22244|3 HOOK HANGER MAG...|      12|2010-12-01 13:04:00|     1.95|   12662.0|Germany|
|   536527|    22243|5 HOOK HANGER RED...|      12|2010-12-01 13:04:00|     1.65|   12662.0|Germany|
|   536527|    47421|ASSORTED COLOUR L...|      24|2010-12-01 13:04:00|     0.42|   12662.0

In [281]:

b = a.groupBy(col("InvoiceNo")).agg(sum(col("UnitPrice")*col("Quantity")).alias("totalcost")).sort(col("totalcost"))
b.show()

+---------+---------+
|InvoiceNo|totalcost|
+---------+---------+
|  C536548|   -122.3|
|   536527|   261.48|
+---------+---------+



In [283]:
b.where(col("totalcost") >10).count()


1

In [302]:
german_cust = retail.select("*",(col("country")=="Germany").alias("bought_in_germany"))
german_cust.show()

+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+-----------------+
|InvoiceNo|StockCode|         Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|bought_in_germany|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+-----------------+
|   536365|   85123A|WHITE HANGING HEA...|       6|2010-12-01 08:26:00|     2.55|   17850.0|United Kingdom|            false|
|   536365|    71053| WHITE METAL LANTERN|       6|2010-12-01 08:26:00|     3.39|   17850.0|United Kingdom|            false|
|   536365|   84406B|CREAM CUPID HEART...|       8|2010-12-01 08:26:00|     2.75|   17850.0|United Kingdom|            false|
|   536365|   84029G|KNITTED UNION FLA...|       6|2010-12-01 08:26:00|     3.39|   17850.0|United Kingdom|            false|
|   536365|   84029E|RED WOOLLY HOTTIE...|       6|2010-12-01 08:26:00|     3.39|   17850.0|United Kingdom|           

In [308]:
#retail.show()
allinvoice = retail.select("*").groupBy(col("invoiceno")).agg(sum(col("UnitPrice")*col("Quantity")).alias("totalcost")).orderBy(col("totalcost"))
allinvoice.show()

+---------+-------------------+
|invoiceno|          totalcost|
+---------+-------------------+
|  C536391|-141.48000000000002|
|  C536548|             -122.3|
|  C536379|              -27.5|
|  C536506|              -25.5|
|  C536383|              -4.65|
|  C536543|               -3.8|
|   536589|                0.0|
|   536547|                0.0|
|   536554|                0.0|
|   536414|                0.0|
|   536553|                0.0|
|   536546|                0.0|
|   536550|                0.0|
|   536545|                0.0|
|   536549|                0.0|
|   536552|                0.0|
|   536555| 2.9699999999999998|
|   536521|               4.95|
|   536568|               5.04|
|   536565|                6.7|
+---------+-------------------+
only showing top 20 rows



JOIN

In [309]:
retail.join(allinvoice,["invoiceno"]).show()

+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+------------------+
|InvoiceNo|StockCode|         Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|         totalcost|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+------------------+
|   536365|   85123A|WHITE HANGING HEA...|       6|2010-12-01 08:26:00|     2.55|   17850.0|United Kingdom|            139.12|
|   536365|    71053| WHITE METAL LANTERN|       6|2010-12-01 08:26:00|     3.39|   17850.0|United Kingdom|            139.12|
|   536365|   84406B|CREAM CUPID HEART...|       8|2010-12-01 08:26:00|     2.75|   17850.0|United Kingdom|            139.12|
|   536365|   84029G|KNITTED UNION FLA...|       6|2010-12-01 08:26:00|     3.39|   17850.0|United Kingdom|            139.12|
|   536365|   84029E|RED WOOLLY HOTTIE...|       6|2010-12-01 08:26:00|     3.39|   17850.0|United Kingdom|    

PIVOT( How many instances of each product were sold in each country?)

In [294]:
A = retail.groupby("stockcode").pivot("country").agg(sum("quantity"))
A.show()

+---------+---------+----+------+-------+-----------+------+--------------+
|stockcode|Australia|EIRE|France|Germany|Netherlands|Norway|United Kingdom|
+---------+---------+----+------+-------+-----------+------+--------------+
|    22728|     null|null|    24|   null|       null|  null|            13|
|    21259|     null|null|  null|   null|       null|  null|             8|
|    21889|     null|  24|  null|   null|       null|  null|            28|
|    22121|     null|null|  null|   null|       null|  null|            14|
|    90022|     null|null|  null|   null|       null|  null|             1|
|   90210B|     null|null|  null|   null|       null|  null|             1|
|    21894|     null|null|  null|   null|       null|  null|             4|
|    21452|     null|null|  null|   null|       null|  null|            12|
|    21249|     null|null|  null|   null|       null|  null|             2|
|    21671|     null|null|  null|   null|       null|  null|             2|
|    22130| 

DRILL DOWN

In [315]:
#spark.sql("""SELECT StockCode, Country ,sum(Quantity) AS Profit FROM df2 GROUP BY ROLLDOWN(StockCode,Country)""")

AnalysisException: Undefined function: 'ROLLDOWN'. This function is neither a registered temporary function nor a permanent function registered in the database 'default'.; line 1 pos 69

Py4JJavaError: An error occurred while calling o2002.save.
: org.apache.spark.SparkException: Job aborted.
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$.write(FileFormatWriter.scala:231)
	at org.apache.spark.sql.execution.datasources.InsertIntoHadoopFsRelationCommand.run(InsertIntoHadoopFsRelationCommand.scala:188)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult$lzycompute(commands.scala:108)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult(commands.scala:106)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.doExecute(commands.scala:131)
	at org.apache.spark.sql.execution.SparkPlan.$anonfun$execute$1(SparkPlan.scala:180)
	at org.apache.spark.sql.execution.SparkPlan.$anonfun$executeQuery$1(SparkPlan.scala:218)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:151)
	at org.apache.spark.sql.execution.SparkPlan.executeQuery(SparkPlan.scala:215)
	at 